In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import torch
from importlib import reload
print(os.getpid())
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares")

4162177


# Load from twist

In [2]:
twist_filename = r'/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF4_4T1Primary16bit_DNAbit/download_from_Twist/MF4-5.csv'
twist_probes = pd.read_csv(twist_filename)

In [3]:
twist_probes

,Pool Name,Insert Length,Insert Sequence,Step,Shipping est
0,mf4-mf5_probe,133,CCCGCAATGGCTGACAACCGGAGGGTTTGTAAGGTGAATATGGGTG...,delivered,2023-12-14
1,mf4-mf5_probe,133,CCCGCAATGGCTGACAACCGTGGGAGGATTGAGATGAGTTTGGGTG...,delivered,2023-12-14
2,mf4-mf5_probe,133,CCCGCAATGGCTGACAACCGGGGTGGTTTAGTGTGTGTTTTTGGGA...,delivered,2023-12-14
3,mf4-mf5_probe,133,CCCGCAATGGCTGACAACCGGGGTGGTTTAGTGTGTGTTTTGAGGG...,delivered,2023-12-14
4,mf4-mf5_probe,133,CCCGCAATGGCTGACAACCGGGGTGGTTTAGTGTGTGTTTTATCGA...,delivered,2023-12-14
...,...,...,...,...,...
40979,mf4-mf5_probe,133,CGGGTTTCGTTGCGCACACCATAGGAAATGGTGGTAGTGTTGGGAT...,delivered,2023-12-14
40980,mf4-mf5_probe,133,CGGGTTTCGTTGCGCACACCATAGGAAATGGTGGTAGTGTTGGGAT...,delivered,2023-12-14
40981,mf4-mf5_probe,133,CGGGTTTCGTTGCGCACACCATAGGAAATGGTGGTAGTGTTGGTGG...,delivered,2023-12-14
40982,mf4-mf5_probe,133,CGGGTTTCGTTGCGCACACCAGAGTGAGTAGTAGTGGAGTTGGGAT...,delivered,2023-12-14


In [5]:
# parse into library
primer_len = 20
primer_2_lib = {}
for _seq in twist_probes['Insert Sequence']:
    if _seq[:primer_len] not in primer_2_lib:
        primer_2_lib[_seq[:primer_len]] = [_seq]
    else:
        primer_2_lib[_seq[:primer_len]].append(_seq)

In [ ]:
# DNA Merfish bits are using primer 6 as forward, so

In [7]:
lib_sequences = primer_2_lib['CGCAAACTGGTGCGGAAGGC']

## Load readouts

In [18]:
from Bio.Seq import reverse_complement

In [13]:
from Bio import SeqIO

In [16]:
readout_filename = r'/lab/solexa_weissman/puzheng/References/Readouts/updated_Stvs.fasta'

def read_fasta_file(file_path):
    records = list(SeqIO.parse(file_path, "fasta"))
    data = {
        "ID": [record.id for record in records],
        "Name": [record.name for record in records],
        "Description": [record.description for record in records],
        "Sequence": [str(record.seq) for record in records]
    }
    df = pd.DataFrame(data)
    return df

In [19]:
readout_df = read_fasta_file(readout_filename)
readout_df['rc_last20'] = [reverse_complement(_s[-20:]) for _s in readout_df['Sequence']]

In [48]:
readout_df

,ID,Name,Description,Sequence,rc_last20
0,Stv_3,Stv_3,Stv_3 old_barcode,ACAAATCCGACCAGATCGGACGATCATGGG,CCCATGATCGTCCGATCTGG
1,Stv_4,Stv_4,Stv_4 old_barcode,CAAGTATGCAGCGCGATTGACCGTCTCGTT,AACGAGACGGTCAATCGCGC
2,Stv_5,Stv_5,Stv_5 old_barcode,AAGTCGTACGCCGATGCGCAGCAATTCACT,AGTGAATTGCTGCGCATCGG
3,Stv_6,Stv_6,Stv_6 old_barcode,CGAAACATCGGCCACGGTCCCGTTGAACTT,AAGTTCAACGGGACCGTGGC
4,Stv_7,Stv_7,Stv_7 old_barcode,ACGAATCCACCGTCCAGCGCGTCAAACAGA,TCTGTTTGACGCGCTGGACG
...,...,...,...,...,...
70,Stv_131,Stv_131,Stv_131 old_barcode,CGCACGCAACCAGCGCGAACCCTGAGCAAA,TTTGCTCAGGGTTCGCGCTG
71,Stv_133,Stv_133,Stv_133 old_barcode,CGCGAGGGCACTCACGCTGGAAGTGATACG,CGTATCACTTCCAGCGTGAG
72,Stv_136,Stv_136,Stv_136 old_barcode,CGGTCACGCAACCGAACCGACGTCGCAAAT,ATTTGCGACGTCGGTTCGGT
73,Stv_145,Stv_145,Stv_145 old_barcode,TAATCCGGGCAAGTCGGTTGGGAGCGTCCC,GGGACGCTCCCAACCGACTT


In [25]:
# probe_2_codebook
probe_2_code = []
for _s in lib_sequences:
    _probe_rds = []
    for _i, _row in readout_df.iterrows():
        if _row['rc_last20'] in _s:
            _probe_rds.append(_row['ID'])
    probe_2_code.append(_probe_rds)

In [28]:
# Read saved file:
saved_library_filename = r'/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF4_4T1Primary16bit_DNAbit/2023_12_3_twist_probes.xlsx'
saved_library = pd.read_excel(saved_library_filename)

In [80]:
saved_library['name'][103]

'Robo4_ENSMUST00000214185.3_shift:2444_GC:63.3_Tm:60.3_specificity:1_isospecificity:0.16_Stv_44:Stv_50::Stv_48'

In [33]:
saved_lib_sequences = list(saved_library['sequence'])

In [36]:
for _s in lib_sequences:
    if _s not in saved_lib_sequences:
        print(_s)

In [37]:
for _s in saved_lib_sequences:
    if _s not in lib_sequences:
        print(_s)

array([ True,  True,  True, ..., False, False, False])

In [112]:
sel_gene = 'Klrk1'
gene_probes = saved_library.loc[np.array([_n.split('_')[0]==sel_gene for _n in saved_library['name']])]
shifts = [int(_n.split('shift:')[1].split('_')[0]) for _n in gene_probes['name']]

In [113]:
probe_readouts = []
for _i, _row in gene_probes.iterrows():
    _seq = _row['sequence']
    _probe_rds = []
    for _i, _row in readout_df.iterrows():
        if _row['rc_last20'] in _seq:
            _probe_rds.append(_row['ID'])
    probe_readouts.append(':'.join(_probe_rds))
gene_probes['shift'] = shifts
gene_probes['real_readouts'] = probe_readouts

/tmp/ipykernel_4162177/3998037216.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_probes['shift'] = shifts
/tmp/ipykernel_4162177/3998037216.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_probes['real_readouts'] = probe_readouts


In [114]:
gene_probes.sort_values(by='shift')

,name,sequence,library,shift,real_readouts
21,Klrk1_ENSMUST00000032252.8_shift:0_GC:43.3_Tm:...,CGCAAACTGGTGCGGAAGGCGTCAGACAAAAGGGCCCGATTACACC...,MF4-4T1_primary_16bit,0,Stv_45:Stv_48:Stv_50
45,Klrk1_ENSMUST00000032252.8_shift:1_GC:43.3_Tm:...,CGCAAACTGGTGCGGAAGGCCTTACACCAGTATCGCGACGTACGCA...,MF4-4T1_primary_16bit,1,Stv_45:Stv_46:Stv_50
22,Klrk1_ENSMUST00000032252.8_shift:9_GC:46.6_Tm:...,CGCAAACTGGTGCGGAAGGCGTCAGACAAAAGGGCCCGATTACACG...,MF4-4T1_primary_16bit,9,Stv_45:Stv_48:Stv_50
69,Klrk1_ENSMUST00000032252.8_shift:10_GC:43.3_Tm...,CGCAAACTGGTGCGGAAGGCCTTACACCAGTATCGCGACGTAAGAT...,MF4-4T1_primary_16bit,10,Stv_45:Stv_46:Stv_48
70,Klrk1_ENSMUST00000032252.8_shift:70_GC:43.3_Tm...,CGCAAACTGGTGCGGAAGGCACACGGCATCACGGTCCGGCTCTTAC...,MF4-4T1_primary_16bit,70,Stv_45:Stv_46:Stv_48
...,...,...,...,...,...
20,Klrk1_ENSMUST00000032252.8_shift:1182_GC:50.0_...,CGCAAACTGGTGCGGAAGGCACACGGCATCACGGTCCGGCTAGGAA...,MF4-4T1_primary_16bit,1182,Stv_46:Stv_48:Stv_50
91,Klrk1_ENSMUST00000032252.8_shift:1184_GC:50.0_...,CGCAAACTGGTGCGGAAGGCACACGGCATCACGGTCCGGCTGGAGG...,MF4-4T1_primary_16bit,1184,Stv_45:Stv_46:Stv_48
92,Klrk1_ENSMUST00000032252.8_shift:1207_GC:50.0_...,CGCAAACTGGTGCGGAAGGCGTCAGACAAAAGGGCCCGATTCTTAC...,MF4-4T1_primary_16bit,1207,Stv_45:Stv_46:Stv_48
43,Klrk1_ENSMUST00000032252.8_shift:1212_GC:50.0_...,CGCAAACTGGTGCGGAAGGCACACGGCATCACGGTCCGGCTACGCA...,MF4-4T1_primary_16bit,1212,Stv_45:Stv_48:Stv_50


In [84]:
gene_2_readouts = {}
probe_2_code = []
for _i, _row in saved_library.iterrows():
    _gene = _row['name'].split('_')[0]
    _seq = _row['sequence']
    _probe_rds = []
    for _i, _row in readout_df.iterrows():
        if _row['rc_last20'] in _seq:
            _probe_rds.append(_row['ID'])
    probe_2_code.append(_probe_rds)
    # gene 2 probe
    if _gene not in gene_2_readouts:
        gene_2_readouts[_gene] = [_c for _c in _probe_rds]
    else:
        for _c in _probe_rds:
            if _c not in gene_2_readouts[_gene]:
                gene_2_readouts[_gene].append(_c)

In [85]:
probe_2_code

[['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_46', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45', 'Stv_48', 'Stv_50'],
 ['Stv_45'

In [67]:
all_readouts = list(np.unique(np.array(list(gene_2_readouts.values()))))
print(all_readouts)

['Stv_29', 'Stv_30', 'Stv_31', 'Stv_32', 'Stv_33', 'Stv_35', 'Stv_36', 'Stv_37', 'Stv_39', 'Stv_40', 'Stv_42', 'Stv_44', 'Stv_45', 'Stv_46', 'Stv_48', 'Stv_50']


In [70]:
gene_2_binary = {}
for _gene, _gene_readouts in gene_2_readouts.items():
    _binary = []
    for _r in all_readouts:
        if _r in _gene_readouts:
            _binary.append('1')
        else:
            _binary.append('0')
    gene_2_binary[_gene] = ''.join(_binary)

In [71]:
gene_2_binary

{'Klrk1': '0000000000001111',
 'Robo4': '0000000000110011',
 'Aldob': '0000000001010101',
 'Foxp3': '0000000010010110',
 'Siglech': '0000000010011001',
 'Cpa3': '0000000001011010',
 'Cd8b1': '0000000000111100',
 'Fzd2': '0000000010100101',
 'Cd79a': '0000000001100110',
 'Wnt7b': '0000000001101001',
 'Ebf1': '0000000011000011',
 'Ldb2': '0000000011001100',
 'C1qb': '0000001100000011',
 'Klrb1c': '0000010100000101',
 'Arhgef37': '0000100100000110',
 'Mat1a': '0000100100001001',
 'Prex2': '0000010100001010',
 'Chil1': '0000001100001100',
 'Cyp4b1': '0001000100010001',
 'Grm6': '0100000100010100',
 'Ms4a1': '1000000100011000',
 'Gata2': '0010000100100001',
 'Ifng': '0001000100100010',
 'Stxbp6': '1000000100100100',
 'Fcmr': '0100000100101000',
 'Ccr9': '0000001100110000',
 'Mrc1': '0100000101000001',
 'Prdm1': '1000000101000010',
 'Skap1': '0001000101000100',
 'Fgfbp1': '0010000101001000',
 'Hoxa10': '0000010101010000',
 'Cd247': '1000000110000001',
 'Mmp12': '0100000110000010',
 'Ibsp': '

In [62]:
import MERFISH_probe_design.IO.file_io as fio

In [63]:
library_master_folder = r'/lab/solexa_weissman/puzheng/MERFISH_Probes'
v1_codebook_filename = os.path.join(library_master_folder, '4T1_MERFISH', 'MF4_4T1Primary16bit_DNAbit','codebook.csv')
codebook_infos = fio.load_merlin_codebook(v1_codebook_filename)


In [74]:
codebook_gene_2_binary = {}
for _i, _row in codebook_infos[3].iterrows():
    codebook_gene_2_binary[_row['name']] = _row['barcode_str']

In [75]:
codebook_gene_2_binary

{'Trdc': '1100000011000000',
 'Ighd': '0000100110010000',
 'Klhl14': '1111000000000000',
 'Tmem252': '1010000010100000',
 'Dab2': '0010001010001000',
 'Camk4': '0000101010100000',
 'Themis': '0100010000100010',
 'Cd200r3': '1001000000001001',
 'Gata2': '0010000100100001',
 'Ifng': '0001000100100010',
 'Cd96': '1100000000110000',
 'Cd8b1': '0000000000111100',
 'Retnlg': '0010010001000010',
 'Cd8a': '0001100010000001',
 'Clec14a': '0101010100000000',
 'Pou2af1': '0000110000000011',
 'Col5a2': '0001100001000010',
 'Wnt7b': '0000000001101001',
 'Ibsp': '0010000110000100',
 'Htr1b': '0001001000100001',
 'Prex2': '0000010100001010',
 'Ankrd1': '0101000000001010',
 'Robo4': '0000000000110011',
 'Cd4': '1000100010001000',
 'Foxp3': '0000000010010110',
 'Prdm1': '1000000101000010',
 'Hpd': '1100001100000000',
 'Shank3': '0100100001001000',
 'Gpihbp1': '0101000010100000',
 'Flt3': '0010010010000001',
 'Cyp11a1': '0000110011000000',
 'Gpr39': '1100000000000011',
 'Prss34': '0010100010000010',
 'S

In [77]:
for _gene in gene_2_binary:
    if gene_2_binary[_gene] != codebook_gene_2_binary[_gene]:
        print(_gene)
    print(_gene, gene_2_binary[_gene], codebook_gene_2_binary[_gene])

Klrk1 0000000000001111 0000000000001111
Robo4 0000000000110011 0000000000110011
Aldob 0000000001010101 0000000001010101
Foxp3 0000000010010110 0000000010010110
Siglech 0000000010011001 0000000010011001
Cpa3 0000000001011010 0000000001011010
Cd8b1 0000000000111100 0000000000111100
Fzd2 0000000010100101 0000000010100101
Cd79a 0000000001100110 0000000001100110
Wnt7b 0000000001101001 0000000001101001
Ebf1 0000000011000011 0000000011000011
Ldb2 0000000011001100 0000000011001100
C1qb 0000001100000011 0000001100000011
Klrb1c 0000010100000101 0000010100000101
Arhgef37 0000100100000110 0000100100000110
Mat1a 0000100100001001 0000100100001001
Prex2 0000010100001010 0000010100001010
Chil1 0000001100001100 0000001100001100
Cyp4b1 0001000100010001 0001000100010001
Grm6 0100000100010100 0100000100010100
Ms4a1 1000000100011000 1000000100011000
Gata2 0010000100100001 0010000100100001
Ifng 0001000100100010 0001000100100010
Stxbp6 1000000100100100 1000000100100100
Fcmr 0100000100101000 0100000100101000


In [73]:
codebook_infos[3]

,name,id,barcode_str
0,Trdc,ENSMUST00000196323.2,1100000011000000
1,Ighd,ENSMUST00000194162.6,0000100110010000
2,Klhl14,ENSMUST00000122333.2,1111000000000000
3,Tmem252,ENSMUST00000057243.6,1010000010100000
4,Dab2,ENSMUST00000080880.12,0010001010001000
...,...,...,...
131,Blank-8,,0000001111000000
132,Blank-9,,0000000010101010
133,Blank-10,,0010001001000100
134,Blank-11,,1100110000000000


In [54]:
gene_2_readouts[_gene]

['Stv_30', 'Stv_31', 'Stv_32', 'Stv_29']